## Create web Database
Short script to create a starting database for the web application.

In [ ]:
# update tweepy to latest version (google colab has an outdated version)
!pip install --upgrade tweepy
# must restart runtime after upgrade

In [ ]:
#nltk.download('punkt')
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [1]:
import pandas as pd
import re
import tweepy
from sqlalchemy import create_engine
import datetime
import time
import joblib

# NLP
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
#nltk.download('omw-1.4')

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load authorities

top25 =pd.read_csv("drive/My Drive/TFG/top25.csv",
                    lineterminator='\n', index_col=0)
display(top25.head())

,PageRank,Hub,Auth,uid,description,protected,followers,tweet_count
UNBiodiversity,0.085260,0.015712,0.117600,40001204,Tweets from the Secretariat of the Convention ...,False,130459,62925
UNDP,0.071354,0.072492,0.091977,20596281,Working to end poverty for good. Transforming ...,False,1776404,73296
UNDP4Youth,0.055212,0.008768,0.020115,3291441743,@UNDP works with & for #Youth worldwide to ach...,False,84584,15109
SEforALLorg,0.053700,0.006222,0.204715,314125926,SEforALL works in partnership with the @UN and...,False,57363,25901
ItalyUN_NY,0.042330,0.001226,0.023236,253137919,Official profile of Italian Mission to the UN ...,False,25638,29835


In [ ]:
# text cleaner:
def text_cleaner(raw_text):
  # Tweet text preleaning
  def tweet_preCleaning(tweet):
      # replace endlines with spaces
      tweet = tweet.replace('\n',' ')
      
      # Remove media links and undisired characters
      return re.sub(r"(@[A-Za-z0–9_]+)|#|http\S+|sdgs?|&\w+|[^\w\s]", '', tweet)

  def stopword_removal(tweet):
      return [w for w in tweet if w not in stopwords.words('english')]

  def tweet_lemmatizing (tweet): # input: list of tokenized words from a tweet
      lemmatizer = WordNetLemmatizer() # lemmatizer instantiation
      return [lemmatizer.lemmatize(word) for word in tweet]
  
  text = tweet_preCleaning(raw_text)
  text = word_tokenize(text.lower())
  text = stopword_removal(text)
  text = tweet_lemmatizing(text)
  return text

In [ ]:
# load bearer token
tokens = open('drive/My Drive/TFG/academic research tokens.txt').readline()
bearer_token = re.search('\w+,\s(.+)\s', tokens).group(1)

In [ ]:
# instantiate API endpoint and authenticate
client = tweepy.Client(bearer_token)

In [ ]:
# final dataframe: (twid, uid, text, created_at) text will be substituted by tag
db = dict()
for uid in top25['uid']:
  # start request
  try:
    for tweets in tweepy.Paginator(client.get_users_tweets, 
                                    uid, 
                                    exclude=['retweets','replies'],
                                    tweet_fields=['created_at','text'],
                                    max_results=100,
                                    start_time= '2022-06-01T00:00:00Z', #less recent
                                    end_time= datetime.datetime.now().replace(microsecond=0).isoformat()+"Z" #most recent
                                    ): 
      # wait 1 sec to avoid getting rate capped
      time.sleep(1)
      

      # store the new data in the dictionary
      if tweets.meta['result_count']>0: # if it has any referenced tweets, then store them
        for tweet in tweets.data:
          db[tweet.id] = {'uid':uid, 'text':tweet.text, 'created_at':tweet.created_at}

  # handling the exceptions
  except tweepy.errors.TweepyException as e:
    print(e)
    if e.args[0][0].code == 429: # Too many requests
      print('Two minute break. Go drink some water.')
      time.sleep(60*2)
    else:
      print('Something has gone wrong. SOS')
      break

In [ ]:
df = pd.DataFrame.from_dict(db, orient='index')
display(df.head())

,uid,text,created_at
1536153629127626752,40001204,New study estimates that protecting 30% of glo...,2022-06-13 01:09:00+00:00
1536077880169578497,40001204,"Ahead of #COP15, @mremae explains what the #po...",2022-06-12 20:08:00+00:00
1536046423044608000,40001204,Promoting and ensuring the safe use of synthet...,2022-06-12 18:03:00+00:00
1536017482598690817,40001204,"When it comes to action #ForNature, the import...",2022-06-12 16:08:00+00:00
1535958342476046336,40001204,"“Here we are, 50 years later, completing exper...",2022-06-12 12:13:00+00:00


In [ ]:
def join_text (tweets):
    return ' '.join([w for w in tweets])

df['clean_text']=df['text'].apply(lambda x: text_cleaner(x))
df['clean_text']=df['clean_text'].apply(lambda x: join_text(x))

In [ ]:
display(df.head())

,uid,text,created_at,clean_text
1536153629127626752,40001204,New study estimates that protecting 30% of glo...,2022-06-13 01:09:00+00:00,new study estimate protecting 30 global land a...
1536077880169578497,40001204,"Ahead of #COP15, @mremae explains what the #po...",2022-06-12 20:08:00+00:00,ahead cop15 explains post2020 global biodivers...
1536046423044608000,40001204,Promoting and ensuring the safe use of synthet...,2022-06-12 18:03:00+00:00,promoting ensuring safe use synthetic biology ...
1536017482598690817,40001204,"When it comes to action #ForNature, the import...",2022-06-12 16:08:00+00:00,come action fornature importance good gender d...
1535958342476046336,40001204,"“Here we are, 50 years later, completing exper...",2022-06-12 12:13:00+00:00,50 year later completing experiment started ba...


In [ ]:
biSVM = joblib.load("drive/My Drive/TFG/biSVM.pkl")
biVectorizer = joblib.load("drive/My Drive/TFG/biSVM_vectorizer.pkl")
SDG17 = joblib.load("drive/My Drive/TFG/SDG17.pkl")
SDG17Vectorizer = joblib.load("drive/My Drive/TFG/SDG17_vectorizer.pkl")

In [ ]:
df['relevant_tag'] = df['clean_text'].apply(lambda x: biSVM.predict(biVectorizer.transform([x]))[0])
df['SDGtag'] = df['clean_text'].apply(lambda x: SDG17.predict(SDG17Vectorizer.transform([x]))[0])

In [ ]:
display(df.head())

,uid,text,created_at,clean_text,relevant_tag,SDGtag
1536153629127626752,40001204,New study estimates that protecting 30% of glo...,2022-06-13 01:09:00+00:00,new study estimate protecting 30 global land a...,SDG,14
1536077880169578497,40001204,"Ahead of #COP15, @mremae explains what the #po...",2022-06-12 20:08:00+00:00,ahead cop15 explains post2020 global biodivers...,SDG,15
1536046423044608000,40001204,Promoting and ensuring the safe use of synthet...,2022-06-12 18:03:00+00:00,promoting ensuring safe use synthetic biology ...,SDG,11
1536017482598690817,40001204,"When it comes to action #ForNature, the import...",2022-06-12 16:08:00+00:00,come action fornature importance good gender d...,SDG,5
1535958342476046336,40001204,"“Here we are, 50 years later, completing exper...",2022-06-12 12:13:00+00:00,50 year later completing experiment started ba...,SDG,4


In [ ]:
# Store database just in case
df.to_csv('drive/My Drive/TFG/web_db_safe.csv')

In [ ]:
# convert to SQLite database:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///drive/My Drive/TFG/webDB.db', echo=False)

web_df = df[['uid','created_at','SDGtag']][df['relevant_tag']=='SDG']

In [ ]:
web_df.to_sql('webDB', con=engine)

## Redo of the database:

In [3]:
df =pd.read_csv("drive/My Drive/TFG/web_db_safe.csv",
                    lineterminator='\n', index_col=0)
display(df.head())

,uid,text,created_at,clean_text,relevant_tag,SDGtag
1536153629127626752,40001204,New study estimates that protecting 30% of glo...,2022-06-13 01:09:00+00:00,new study estimate protecting 30 global land a...,SDG,14
1536077880169578497,40001204,"Ahead of #COP15, @mremae explains what the #po...",2022-06-12 20:08:00+00:00,ahead cop15 explains post2020 global biodivers...,SDG,15
1536046423044608000,40001204,Promoting and ensuring the safe use of synthet...,2022-06-12 18:03:00+00:00,promoting ensuring safe use synthetic biology ...,SDG,11
1536017482598690817,40001204,"When it comes to action #ForNature, the import...",2022-06-12 16:08:00+00:00,come action fornature importance good gender d...,SDG,5
1535958342476046336,40001204,"“Here we are, 50 years later, completing exper...",2022-06-12 12:13:00+00:00,50 year later completing experiment started ba...,SDG,4


In [4]:
!pip install flask-sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///drive/My Drive/TFG/webDB.db'
db = SQLAlchemy(app)

/usr/local/lib/python3.7/dist-packages/flask_sqlalchemy/__init__.py:873: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [6]:
#(twid, uid, tag, created_at)
class Tweet(db.Model):
    id = db.Column(db.BigInteger, primary_key=True)
    auth_id = db.Column(db.Integer, nullable=False)
    tag = db.Column(db.Integer, nullable=False)
    created_at = db.Column(db.DateTime, nullable=False)

    def __repr__(self):
        return "Tweet({}, {}, {})".format(self.id, self.auth_id, self.tag)

In [7]:
# create the database
db.create_all()

In [8]:
# filter the dataset:
web_df = df[['uid','created_at','SDGtag']][df['relevant_tag']=='SDG']
display(web_df.head())

,uid,created_at,SDGtag
1536153629127626752,40001204,2022-06-13 01:09:00+00:00,14
1536077880169578497,40001204,2022-06-12 20:08:00+00:00,15
1536046423044608000,40001204,2022-06-12 18:03:00+00:00,11
1536017482598690817,40001204,2022-06-12 16:08:00+00:00,5
1535958342476046336,40001204,2022-06-12 12:13:00+00:00,4


In [9]:
web_df['created_at']=pd.to_datetime(web_df['created_at'])
web_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1153 entries, 1536153629127626752 to 1531941539945099264
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   uid         1153 non-null   int64              
 1   created_at  1153 non-null   datetime64[ns, UTC]
 2   SDGtag      1153 non-null   int64              
dtypes: datetime64[ns, UTC](1), int64(2)
memory usage: 36.0 KB


In [10]:
#db.drop_all()

In [10]:
# populate the database
def populate(tweet):
  db.session.add(Tweet(id=tweet.name, auth_id=tweet.uid, created_at=tweet.created_at, tag=tweet.SDGtag))
  return

web_df.apply(lambda x: populate(x), axis=1)

1536153629127626752    None
1536077880169578497    None
1536046423044608000    None
1536017482598690817    None
1535958342476046336    None
                       ... 
1532121072925016064    None
1532075507969363968    None
1532030475270819842    None
1531956366440538118    None
1531941539945099264    None
Length: 1153, dtype: object

In [11]:
Tweet.query.first()

Tweet(1536153629127626752, 40001204, 14)

In [12]:
# store changes
db.session.commit()